# Exploratory notebook 
The goal of this notebook is to conduct the research part of "predicting the results of a match". Importing data from past seasons, transforming that data to extract new indicators (prétraitement et EDA), developping and evaluating models.

In [ ]:

import pandas as pd             # manipulation de données tabulaires
import sys
import os
import json                     # pour afficher proprement le JSON si besoin
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(".."))


In [ ]:
from src.etl import fetch_matches, save_raw_matches

In [ ]:
matches_json=fetch_matches(2015,2024) #Ligue 1 saison 2024-2025

save_raw_matches(matches_json)